In [ ]:
import mlflow
import mlflow.pyfunc
from datetime import datetime


# model registration 
from mlflow.models.resources import DatabricksServingEndpoint

catalog_name = "catalog_name" ## PLACEHOLDER
schema_name = "schema_name" ## PLACEHOLDER
agent_name = "agent_name" ## PLACEHOLDER
model_name = f"{catalog_name}.{schema_name}.{agent_name}"
model_version = "1.0.0"

def register_model_to_mlflow():
  try:
      if mlflow.active_run() is not None:
          mlflow.end_run()
      
      mlflow.set_registry_uri("databricks-uc")

      with mlflow.start_run(run_name = f"register_{model_name}_{datetime.now().strftime('%Y%m%d_%H%M%S')}") as run:
          
          model_info = mlflow.pyfunc.log_model(
              python_model = "mlflow_agent_wrapper.py", ## PLACEHOLDER
              artifact_path = "agent", ## PLACEHOLDER
              registered_model_name = model_name,
              code_paths = ['agent.py','utilities.py','database_schema.py'], ## PLACEHOLDER
              pip_requirements = "requirements.txt", ## PLACEHOLDER
              resources = [DatabricksServingEndpoint(endpoint_name="databricks-claude-sonnet-4")] ## PLACEHOLDER
          )

  except Exception as e:
        print(f"Model registration failed: {str(e)}")
        return {'status':'error','error':str(e)}

registration_result = register_model_to_mlflow()

In [ ]:
# deploy the model as a serving endpoint
from mlflow.tracking import MLflowClient
from databricks import agents

def get_latest_model_version(model_name):
  mlflow_client = MLflowClient(registry_uri="databricks-uc")
  latest_version = 1
  for mv in mlflow_client.search_model_versions(f"name='{model_name}'"):
      version_int = int(mv.version)
      if version_int > latest_version:
         latest_version = version_int
  return latest_version

endpoint_name = 'placeholder'

agents.deploy(
    model_name=model_name,
    model_version=get_latest_model_version(model_name),
     endpoint_name = endpoint_name,
     scale_to_zero_enabled = True
     )